# Photometry in Python - Photutils, DAOFIND, and Aperture Photometry

This notebook is based on a series created by Stuart Littlefair at the University of Sheffield. 
It also makes use of 'phot_helpers.py', a set of useful scripts created by Stuart Littlefair for photometry.


## Steps

At the end of this notebook you should be able to measure magnitudes for stars in your calibrated Nickel images. 

The steps are:
1. Finding the stars in our image;
2. Measuring instrumental magnitudes (photometry);
3. Determining an astrometric solution using astrometry.net;
4. Matching stars to an all-sky survey to determine photometric zeropoints;
5. Applying the calculated zeropoints to transform between instrumental and calibrated magnitudes;
6. Plotting the results in a colour-magnitude diagram.

# Setting up Astrometry.net

We will use astrometry.net to determine an astrometric solution for our images. 

An astrometric solution will allow us to transform pixel (X-Y) co-ordinates to RA-Dec co-ordinates for all of the stars in our image. 

Astrometry.net provide the source code, so we could build the application oursevles, and use their index images to determine our astrometric solution. But in practice, it's much easier to use their web service for our purposes.

To use astrometry.net's web-based servives we need an API key. The API key is a unique identifier telling astrometry.net who you are. 

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<font color='green'>
<h2><span class="fa fa-pencil"></span> Set up astrometry.net API </h2>
</font>
**Create an account with astrometry.net:**
https://nova.astrometry.net/

**Then navigate to the 'API' tab. 
Copy your API key and add it below. Or you can add it to the 'astroquery.cfg' file, then move this file to ~/.astropy/config/**
    
</div>
</section>

In [ ]:
from astroquery.astrometry_net import AstrometryNet
ast = AstrometryNet()
#ast.api_key = '##########'

# Photometry with Photutils

Many tools exist to perform photometry (IRAF, GAIA, AstroImageJ etc.)
We are going to be using the Python package [photutils](http://photutils.readthedocs.org/en/latest/) to do most of the heavy lifting for us.

If you do not have photutils already installed, the code cell below will install it. Run this cell, and if there are any **errors**, ask for help (warnings are ok/expected).

In [ ]:
import sys
!{sys.executable} -m pip install photutils

In [ ]:
## if this code cell runs without error, you have successfully installed photutils!
import photutils as p

# Functions for Aperture Photometry

Now we'll make a couple of functions to make life easier later. 

The first will perform aperture photometry. The second will plot the full-width at half-maximum for a star.

In [ ]:
# A function to perform aperture photometry
from photutils.utils import calc_total_error
from photutils.aperture import CircularAperture
from photutils.aperture import CircularAnnulus
import astropy.stats as st
import warnings
from astropy.wcs import WCS
def aperture_photometry(data, header, sources, aperture_radius, sky_inner_radius, sky_outer_radius):
    """
    Calculate Aperture Photometry on a list of sources

    Parameters
    ----------
    data:  `np.ndarray`
        A 2D array of pixel values of your data. From a FITS file, you can create this array with
        `fits.getdata`.

    header: `~astropy.fits.Header`
        A FITS Header object. From a FITS file, you can create this object with
        `fits.getheader`.

    sources: `~astropy.table.Table`
        A table of detected sources for the image. Usually, `photutils.DAOStarFinder` would be
        used to create this list.

    aperture_radius: float
        Radius of the target aperture, in pixels

    sky_inner_radius: float
        Radius of the inner aperture that makes up the sky annulus

    sky_outer_radius: float
        Radius of the outer aperture that makes up the sky annulus

    Returns
    -------
    phot_table: `~astropy.table.Table`
        A table of measurements for each source, including instrumental magnitude and error.
    """
    # make apertures around sources, and annuli for sky estimation
    positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
    apertures = CircularAperture(positions, r=aperture_radius)
    sky_annulus = CircularAnnulus(positions,
                                    r_in=sky_inner_radius, r_out=sky_outer_radius)
    annulus_masks = sky_annulus.to_mask(method='center')

    # aperture photometry - calculates total counts in apertures, with errors
    # calc_total_error uses CCD SNR equation 
    error_arr = calc_total_error(data, 12.0, egain)
    phot_table = p.aperture.aperture_photometry(data, apertures, error=error_arr)

    # calculate the Sky background. Because the sky annulus might have other
    # stars inside it, we will take a CLIPPED MEAN of the counts in the annulus
    # to try and reject the contribution from stars.
    bkg_mean = []
    for mask in annulus_masks:
        annulus_data = mask.multiply(data)
        annulus_data_1d = annulus_data[mask.data > 0]
        mean_sigclip, _, _ = st.sigma_clipped_stats(annulus_data_1d)
        bkg_mean.append(mean_sigclip)

    # now we know the mean sky counts. We multiply by ratio of annulus area to
    # target aperture area. This gives expected number of sky counts in target
    # aperture.
    bkg_mean = np.array(bkg_mean)
    phot_table['sky_mean'] = bkg_mean
    phot_table['aper_bkg'] = bkg_mean * apertures.area
    phot_table['aper_sum_bksub'] = phot_table['aperture_sum'] - phot_table['aper_bkg']

    # Tricky bit now! We want to know what Right Ascension and Declination
    # each pixel corresponds to. We will use the information in the FITS header
    # (the so-called "World Coordinate System") to work this out. 
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        wcs = WCS(header)
    ra, dec = wcs.all_pix2world(phot_table['xcenter'], phot_table['ycenter'], 0)
    phot_table['RA'] = ra
    phot_table['DEC'] = dec

    # Calculate Instrumental Magnitude and Error
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        phot_table['instrumental_mag'] = -2.5*np.log10(phot_table['aper_sum_bksub'] / header['EXPTIME'])
    phot_table['e_instrumental_mag'] = phot_table['aperture_sum_err'] / phot_table['aper_sum_bksub']

    return phot_table

In [ ]:
# We will use this function later to measure FWHM in our images
def measure_FWHM(data, sources):
    # averagely bright stars
    lims = np.percentile(sources['flux'], (60, 70))
    mask = reduce(
        np.logical_and,
        (sources['flux'] > lims[0], sources['flux'] < lims[1],
         sources['xcentroid'] > 15, sources['ycentroid'] > 15,
         sources['xcentroid'] < data.shape[1]-15, 
         sources['ycentroid'] < data.shape[0]-15
        )
    )

    positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
    tree = KDTree(positions)
    # nearest neighbours
    dist, ind = tree.query(positions, 2)

    # most isolated star of proper brightness
    idx = dist[:, 1][mask].argmax()
    location = positions[mask][idx]
    cutout = Cutout2D(data, location.T, 15)

    fig, axis = plt.subplots(nrows=1, ncols=2, figsize=(13, 6))
    norm = ImageNormalize(cutout.data, interval=AsymmetricPercentileInterval(1, 99))
    axis[0].imshow(cutout.data, cmap='Greys', origin='lower', norm=norm, interpolation='nearest')
    axis[0].set_xlabel('X')
    axis[0].set_ylabel('Y')

    xc, yc = centroid_com(cutout.data)
    x = np.arange(15) - xc
    y = np.arange(15) - yc
    X, Y = np.meshgrid(x, y)
    R = np.sqrt(X**2 + Y**2)
    axis[1].plot(R.ravel(), cutout.data.ravel(), '.')
    axis[1].set_xlabel('Distance from centre of star')
    axis[1].set_ylabel('Counts')


# Workflow

The basic steps that we will work through in this workbook are:
1. [Read in an image](#ReadImage)
2. [Indentify stars in the image](#SourceList)
3. [Measure the brightness of the stars in that image (aperture photometry)](#ApPhot)
4. [Repeat the above steps for other filters](#BFilt)
5. Determine photometric zeropoints using standard stars
6. Apply the photometric zeropoints to your measured magnitudes

<a id='ReadImage'></a>

# Read in your image

Using the code cell below, read in one of your pre-calibrated Nickel images with plenty of stars. A good choice would be a long exposure V-band image. We're going to read the image into an array called `data` and read the header from the FITS file into a variable called `header`. 

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<font color='green'>
<h2><span class="fa fa-pencil"></span> Choose the image to use </h2>
</font>   
    
**Set the input file (ifile) that you want to perform photometry on.**
    
</div>
</section>


In [ ]:
from astropy.io import fits,ascii
import numpy as np
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization import AsymmetricPercentileInterval
from matplotlib import pyplot as plt
from functools import reduce
from scipy.spatial import KDTree
from astropy.nddata import Cutout2D
from photutils.centroids import centroid_com

# We need to set the gain of our CCD to allow us to measure errors correctly later
# For the Nickel, gain is 2 e/ADU
egain = 2.0
ifile = '/home/jrees/DataReduction/20230720/Standards/d1084_os_bs_ff_bp_crj.fits'
data, header = fits.getdata(ifile,header=True)


<a id='SourceList'></a>
# Creating a Source List

The first step is to detect our sources. We will do this using an algorithm called DAOFIND ([Stetson 1987, PASP, 99, 191](https://ui.adsabs.harvard.edu/abs/1987PASP...99..191S/abstract)). DAOFIND looks for bright regions in the image that have a peak brightness greater than some threshold and that have a size and shape similar to a Gaussian of specified FWHM.

Stars in our image will stand out above the background, and DAOFIND will find them, but we need to know what threshold to use. One way of doing this is to measure the statistics of the **background** in our image. If we measure the average value of the background, and the amount the background varies, we can look for regions that are significantly brighter than background pixels. 

Below we will do that using a "sigma-clipped" mean - this estimates the average background and the standard deviation. We then throw away all the pixels more than 3 standard deviations (sigma) away from the mean, and repeat the process. We carry on until no pixels are more than 3 standard deviations away from the average value, then calculate the mean, median and standard deviation of the remaining pixels.

You may want to tweak this value of sigma until you're happy with the sky level.

In [ ]:
# import sigma_clipping function from astropy
from astropy.stats import sigma_clipped_stats

mean_background, median_background, background_standard_deviation = sigma_clipped_stats(data, sigma=3.0)

print("The background has an average value of {:.1f} and a standard deviation of {:.1f} counts".format(
    mean_background, background_standard_deviation))

# Finding Stars

Now we know how bright our background is, and how much it varies, let's look for stars that are brighter than the background plus 5 standard deviations. That should be enough that we don't identify bright background pixels as stars by accident. The DAOFIND algorithm needs a guess for how big the stars are - as a Gaussian FWHM - we'll guess at 6 pixels for now, but you can tweak this further.

In [ ]:
from photutils.detection import DAOStarFinder

# Set the fwhm initial guess
fwhm = 6.0
# make a star finder object to look for stars with FWHM~6 pixels that are more than 5-sigma above background
daofind = DAOStarFinder(fwhm=fwhm, threshold=5*background_standard_deviation)

# Mask the data to remove the edges/corners
mask=np.ma.make_mask(data,copy=True,shrink=True,dtype=bool)
mask[:,:]=False
mask[950:,0:50]=True
mask[0:50,0:50]=True
mask[:,1010:]=True

# use it to find stars. We'll subtract the background off first, so background pixels have an average value of 0
sources = daofind(data - median_background,mask=mask)

print(sources)

The ``sources`` variable contains a table of all the detected stars. There are various columns, but the ones we are interested in is the X and Y positions of the stars, which you can find with ```sources['xcentroid']``` and ```sources['ycentroid']```.

But how do we know we've found most of the stars? Or if we are mistakenly identifying bright background pixels as stars? We can inspect our sources by-eye. To make this easier, we're going to use one of the handy functions written by Stuart Littlefair 


In [ ]:
# How big should we make the red apertures (in pixels)?
radius=fwhm
# Set up the positions of stars (from DAOFIND) and the size of our apertures
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=radius)
# Normalise the image
norm = ImageNormalize(data, interval=AsymmetricPercentileInterval(5, 95))

fig = plt.figure(figsize=(15, 12))
apertures.plot(color='red', lw=1.5, alpha=0.5)
plt.imshow(data, cmap='Greys', origin='lower', norm=norm, interpolation='nearest')


# Tweak the detection settings </h2>
   
The DAOFIND algorithm requires a threshold for star detection, and a typical FWHM of the stars in the images. Try different settings for these values, and see how they affect the detection of stars in your data. Make a decision about what values to use for this image.
    


# FWHM of stars in the image

We will also need an estimate for the FWHM of stars in the image. We can estimate this by-eye by plotting a bright, isolated, star and plotting the brightness against distance from the star's centre. 

Again, a handy tool exists to do this already. 

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<font color='green'>
<h2><span class="fa fa-pencil"></span> Measure the FWHM of an isolated star </h2>
</font>    
    
**Using the plot below, figure out your FWHM in pixels. You can then go back and update the value of fwhm. 
Using the Nickel plate scale you can convert it to arcseconds (you do remember the value right?)**
    
</div>
</section>


In [ ]:
measure_FWHM(data, sources)

# Aperture Photometry

So, we have a list of detected sources and positions in the image, and an idea of the FWHM of stars in the image. Now we can perform aperture photometry on all of these sources. 
In brief the function performs the following steps:

1. Add up the counts from each source within a target aperture
2. Measure the sky brightness around each source using the sky annulus
3. Subtract the sky contribution from the counts in step 1.
4. Calculate instrumental magnitude from the counts and exposure time.


<a id='ApPhot'></a>
# Perform Aperture Photometry

You'll need to pick values for the aperture radii. 

Target apertures want to be big enough to accept a decent fraction of the flux, but not so large that the measurements are very noisy, or contaminated by nearby stars. As a rule of thumb this aperture might have a radius of 1.5-2x the FWHM.

Sky Annuli want to be wide enough to accurately measure the sky, but not so large that the annuli overlap nearby stars.

To get started, try values around 15, 25, 35 pixels for these apertures. We can tweak them later on.

If you want to view extra information about what 'aperture_photometry' is doing, you can do:

help(aperture_photometry)

In [ ]:
Vmag_inst = aperture_photometry(data, header, sources, 15, 25, 35)


# Photometric Calibration

Now we have a table of instrumental magnitudes (and much besides) in it. The next step is photometric calibration. As a reminder, this involves finding the average value of the *difference* between the calibrated and instrumental magnitudes for all our stars.

There are a couple of options for determining our photometric calibration. Traditionally, we would observe 'standard stars', stars with well determined magnitudes. By observing these standard stars over a range of airmass we can determine the effect of atmospheric extinction, and determine photometric zeropoints as a function of airmass. 

This is a pretty involved process, and probably outside the scope of our data reduction activity this afternoon (though I have created the bare bones of an example in the nickel-standards notebook. 

With the advent of modern all-sky surveys we can also 'cheat' slightly, and use existing photometry from these all-sky surveys to provide our photometric calibrations. 

We will use the [APASS](https://www.aavso.org/apass) catalog; a catalog which combines several other sky surveys to provide data in many filters across much of the sky. Crucially, in this case it includes B, V, r, and i magnitudes, the filters used for our photometry (the r/i filters are not an exact match for our R and I filters, but close enough for this demonstration).

To perform the cross-matching we will use the [astroquery](https://astroquery.readthedocs.io/en/latest/) Python library. 

To perform the cross-match, we'll need to determine an astrometric solution for our data. The astrometric solution will allow us to convert x-y pixel co-ordinates to RA-Dec co-ordinates. 

In [ ]:
from astropy import units as u
from astroquery.xmatch import XMatch

Our photometry is stored as an astropy table, which is a handy object for reading and writing tabular data. These astropy tables play nicely with Jupyter notebooks, so you can simply type the name of the table in a code cell to see the table displayed in the browser.

In [ ]:
Vmag_inst

# Astrometric Solution

To determine our astrometric solution we will make use of the Astrometry.net API. 

We already set up the API key earlier in this notebook, so now we can use solve_from_source_list to determine an astrometric solution from our source list.

In [ ]:
from astropy.table import Table

# Sort sources in ascending order
sources.sort('flux')
# Reverse to get descending order
sources.reverse()

image_width = 1024
image_height = 1024
wcs_header = ast.solve_from_source_list(sources['xcentroid'], sources['ycentroid'],
                                        image_width, image_height,
                                        solve_timeout=120)
print("Finished astrometric solve")
print("-----------------------------------------------------")


Now we need to apply the astrometric solution to our source table. We will replace the existing RA, DEC fields with new values based on the new solution.

In [ ]:
w = WCS(wcs_header)
Vmag_inst['RA'], Vmag_inst['DEC'] = w.wcs_pix2world(Vmag_inst['xcenter'],Vmag_inst['ycenter'],1)


Our source table now has the measured centres of our stars in RA and Dec (**RA** and **DEC**) and the instrumental magnitude and uncertainty (**instrumental_mag** and **e_instrumental_mag**). Note that the magnitude uncertainty is calculated using the CCD signal-to-noise equation.

# Crossmatching with APASS (or some other catalogue)


 We need to calibrate our photometry, which will involve comparing our instrumental magnitudes to calibrated magnitudes measured for the same stars. We need to match our detected stars with those catalogued in the sky survey [APASS](https://www.aavso.org/apass). We are looking for stars who's RA and Dec matches to within some radius. A service called **Vizier** hosts online versions of astronomical catalogs, and we can use the ```Xmatch.query``` function to match an astropy table with a table hosted by Vizier using the code below.

 Run the code cell below, and note carefully how we specify the columns that contain RA and Dec in our *local* table, and how we set the maximum distance for a valid match. **II/336/apass9** is the name of the APASS catalog on Vizier. If you need to find the names of other catalogs (perhaps APASS doesn't cover the patch of sky containing your open cluster), you can enter the catalog name in the search box [here](http://vizier.u-strasbg.fr)
 Something like UCAC4 or UCAC5 or GAIA might be good alternative choices

 **The code cell below may take a while to run. Be patient...**
 

In [ ]:
xmatch = XMatch.query(cat1=Vmag_inst, cat2='vizier:II/336/apass9', max_distance=2*u.arcsec, colRA1='RA', colDec1='DEC')


In [ ]:
xmatch

# Plot the difference and find the zero-point

 Plot the difference between instrumental and calibrated magnitude found above, against the calibrated V-band magnitude on the X-axis. You should see something like the figure below:

<img src="images/V_zeropoint.png" style="margin: 0px" width=750px/>

In [ ]:
delta_mag = xmatch['Vmag'] - xmatch['instrumental_mag']

import matplotlib.pyplot as plt
plt.figure(figsize=(20,12))
plt.plot(xmatch['Vmag'],delta_mag, 'bo')
plt.xlabel('V Mag (APASS)')
plt.ylabel('Delta V (APASS - Inst)')
plt.show

The magnitude difference between instrumental and calibrated magnitude (your Y-axis) *should* be a constant, which is the value of $kX + m_{\rm zp}$. In the figure above, you can see there are plenty of outlying points, and the bright stars deviate from the constant. The outliers are either stars whose photometry is bad (poor sky estimation, or contaminated by very close stars), or spurious detections (i.e not stars). The deviation of the bright stars is caused because they are saturated, and so we cannot accurately measure their flux.

Since $m = m_i - kX + m_{\rm zp}$, we can find the value of $-kX + m_{\rm zp}$. - which I'll call the *zeropoint* from now on - by calculating the **median** difference between the instrumental and calibrated magnitude. The median will be robust against the outliers - but we only want to do it for the non-saturated stars!

# Calculate the zero-point

> Calculate the median value of ```delta_mag```. **If you have evidence for saturated stars**, you can use a NumPy *mask* to only calculate the median of stars that are not saturated 

> The median value of ```delta_mag``` is our estimate of $-kX + m_{\rm zp}$ - i.e the value we want to add to our instrumental magnitudes to get a calibrated magnitude. You can add this value to the **instrumental_mag** column of the VMag_inst table easily using ```VMag_inst['calibrated_mag'] = VMag_inst['instrumental_mag'] + zp```, where ```zp``` is the median value you found.

<section class="challenge panel panel-success"> 
<div class="panel-heading">
<font color='green'>
<h2><span class="fa fa-pencil"></span> Determine your zeropoint mask</h2>
</font>    

**Set the plotmask below to exclude any saturated stars.**
    
**Alternatively, if the crossmatch fails (usually due to a lack of stars) you can adopt 22.5 for V and 23.5 for B**

</div>
</section>


In [ ]:
# First, just calculate the median:

zp = np.median(delta_mag)
print ('zeropoint = ', zp)

# We could also use a mask to exclude saturated points. Replace 21.5 and 21.0 
# with the upper/lower bounds of your 'good' data
plotmask = (delta_mag > 22.4) #& (delta_mag > 21.0)
delta_masked = delta_mag[plotmask]
zp1 = np.median(delta_masked)
print('Masked zeropoint = ',zp1)

In [ ]:
zeropoint_v = zp1
# Now apply the zeropoint to the instrumental magnitudes
Vmag_inst['calibrated_mag'] = Vmag_inst['instrumental_mag'] + zeropoint_v

---------
# Other filters and making a CMD

If we repeat the above steps for other images, we can measure our stellar magnitudes in all of our other filters

<a id='BFilt'></a>
# The B-band data

 Assuming that you started with V, repeat the steps for your B-band data. Make a plot of the difference between instrumental B-band magnitude and APASS B-band magnitude. Calculate the offset needed to correct your instrumental mags (zeropoint).

 Finally, add your zeropoint to the **instrumental_mag** column of the B-band table to make a new **calibrated_mag** column.

In [ ]:
# Grab the data
ifile = '/home/jrees/DataReduction/20230720/Standards/d1080_os_bs_ff_bp_crj.fits'
data, header = fits.getdata(ifile,header=True)

# Measure the background
mean_background, median_background, background_standard_deviation = sigma_clipped_stats(data, sigma=3.0)

print("The background has an average value of {:.1f} and a standard deviation of {:.1f} counts".format(
    mean_background, background_standard_deviation))

# Find the stars 
daofind = DAOStarFinder(fwhm=fwhm, threshold=5*background_standard_deviation)
sources = daofind(data - median_background)
print(sources)

# Check everything looks good
radius=fwhm
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=radius)
norm = ImageNormalize(data, interval=AsymmetricPercentileInterval(5, 95))
fig = plt.figure(figsize=(15, 12))
apertures.plot(color='red', lw=1.5, alpha=0.5)
plt.imshow(data, cmap='Greys', origin='lower', norm=norm, interpolation='nearest')

# Take a look at the FWHM in this image
measure_FWHM(data, sources)

# Time for photometry
Bmag_inst = aperture_photometry(data, header, sources, 15, 25, 35)



# Astrometry round 2

Let's repeat the astrometric solve for this second image

In [ ]:
# Sort sources in ascending order
sources.sort('flux')
# Reverse to get descending order
sources.reverse()

image_width = 1024
image_height = 1024
wcs_header = ast.solve_from_source_list(sources['xcentroid'], sources['ycentroid'],
                                        image_width, image_height,
                                        solve_timeout=120)
print("Finished astrometric solve")
print("-----------------------------------------------------")

w = WCS(wcs_header)
Bmag_inst['RA'], Bmag_inst['DEC'] = w.wcs_pix2world(Bmag_inst['xcenter'],Bmag_inst['ycenter'],1)

# Crossmatch with APASS

Perform the APASS crossmatch for your second filter

In [ ]:
xmatch2 = XMatch.query(cat1=Vmag_inst, cat2='vizier:II/336/apass9', max_distance=2*u.arcsec, colRA1='RA', colDec1='DEC')
xmatch2

delta_mag = xmatch2['Bmag'] - xmatch2['instrumental_mag']

import matplotlib.pyplot as plt
plt.figure(figsize=(20,12))
plt.plot(xmatch2['Bmag'],delta_mag, 'bo')
plt.xlabel('B Mag (APASS)')
plt.ylabel('Delta B (APASS - Inst)')
plt.show

In [ ]:
# First, just calculate the median:

zp = np.median(delta_mag)
print ('zeropoint = ', zp)

# We could also use a mask to exclude saturated points. Replace 21.5 and 21.0 
# with the upper/lower bounds of your 'good' data
plotmask = (delta_mag > 23.4) #& (delta_mag > 21.0)
delta_masked = delta_mag[plotmask]
zp2 = np.median(delta_masked)
print('Masked zeropoint = ',zp2)

zeropoint_b = zp2
# Now apply the zeropoint to the instrumental magnitudes
Bmag_inst['calibrated_mag'] = Bmag_inst['instrumental_mag'] + zeropoint_b

In [ ]:
Bmag_inst

# Crossmatching our calibrated magnitudes

Now you should have a two astropy tables: One with calibrated V-band magnitudes and one with calibrated B-band magnitudes. We need to cross-match these tables with each other, to find which stars in the V-band table match with stars in the B-band table. We can't use ```astroquery```'s Xmatch for this, since they are both local tables. Instead, we will use the ```SkyCoord``` object from astropy, which is meant to work with coordinates on the sky. 



In [ ]:
from astropy.coordinates import SkyCoord
# Create a SkyCoord object for your first filter
coo_v = SkyCoord(Vmag_inst['RA'], Vmag_inst['DEC'], unit=u.deg)

# And for the second filter 
coo_b = SkyCoord(Bmag_inst['RA'], Bmag_inst['DEC'], unit=u.deg)

# match every entry in coo_v with the nearest entry in coo_b to identify the same stars in each catalogue
idx, distance_2d, distance_3d = coo_v.match_to_catalog_sky(coo_b)



In [ ]:
# using idx as a slice for the Bmag_inst table will sort it so that Bmag_inst[0] is the closest match to Vmag_inst[0]
Bmag_inst = Bmag_inst[idx]

# And finally we have to ensure the closest match is actually useful. 
# Some of the 'closest matches' might actually be very far away. 
dist_mask = distance_2d < 3 * u.arcsec
Bmag_inst = Bmag_inst[dist_mask]
Vmag_inst = Vmag_inst[dist_mask]


# Plotting a CMD 

 If we want to plot a CMD from our data, we can extract the calibrated magnitudes, calculate ```B-V``` and plot a colour-magnitude diagram of ```B-V``` against ```V```. Your plot should look something like the one below, which is for the cluster NGC 7789.

<img src="./images/CMD.png" style="margin: 0px" width=750px/>


In [ ]:
B = Bmag_inst['calibrated_mag']
V = Vmag_inst['calibrated_mag']

plt.figure(figsize=(20,12))
plt.plot(B-V,V, 'bo')
plt.xlabel('B-V')
plt.ylabel('V Mag')
plt.gca().invert_yaxis()
plt.show

# Other filters

If you have data in other filters (and time!) you can go ahead and repeat the above steps for the other filters.


In [ ]:
# YOUR CODE HERE
#R
# Grab the data


# Measure the background

# Find the stars 

# Check everything looks good

# Take a look at the FWHM in this image

# Time for photometry

# Astrometric solution

# Match against APASS

# Caluclate the zeropoint

# Apply the zeropoint



In [ ]:
# YOUR CODE HERE
#I
# Grab the data


# Measure the background

# Find the stars 

# Check everything looks good

# Take a look at the FWHM in this image

# Time for photometry

# Astrometric solution

# Match against APASS

# Caluclate the zeropoint

# Apply the zeropoint
